In [84]:
import random

# 一共有多少种类型的牌
pai_types = {'WAN_1':1 , 'WAN_2':2, 'WAN_3':3, 'WAN_4':4,'WAN_5':5,'WAN_6':6, 'WAN_7':7, 'WAN_8':8, 'WAN_9':9,
                 'DONGFENG':10, 'NANFENG':11, 'XIFENG':12, 'BEIFENG':13, 'BAIBAN':14, 'FACAI':15,  'HONGZHONG':16}

# 生成一副牌
pais = [v for v in  list(pai_types.values())  for i in range (0, 4) ]

# 打乱顺序
random.shuffle(pais)

# 牌分成两部分，一部分是手上的牌，一部分是从对家碰、吃得到的不能再改变
host_dynamic_pais = [pais.pop() for i in range(0, 13)]
oppo_dynamic_pais = [pais.pop() for i in range(13, 26)]

host_static_pais = []
oppo_static_pais = []

# 庄家和玩家出的所有牌
host_output_pais = []
oppo_output_pais = []

# 判断谁先摸牌，TRUE： 代表庄家， FALSE：代表对家
oppo_turn = random.random() >= 0.5
if oppo_turn:
    mopai(oppo_dynamic_pais)
    p = chu_pai(oppo_dynamic_pais)
    oppo_dynamic_pais.remove(p)
    oppo_output_pais.append(p)
else:
    mopai(host_dynamic_pais)
    p = chu_pai(host_dynamic_pais)
    host_dynamic_pais.remove(p)
    host_output_pais.append(p)
oppo_turn = not oppo_turn

# 表示游戏还没有完成
not_finished = True

# 依次进行出牌，直到乙方赢了或没有牌了
while not_finished and len(pais) != 0:
    if(oppo_turn):
        """
        对家出牌先判断能能否碰、吃、杠、胡，如果都不满足则进行摸牌，后再进行出牌
        """

        # 最后一次出牌
        last_pai = host_output_pais[-1]
        
        if is_gang(oppo_dynamic_pais, last_pai):# 杠
            gang(oppo_dynamic_pais, oppo_static_pais, last_pai)
        elif is_peng(oppo_dynamic_pais, last_pai): # 碰
            peng(oppo_dynamic_pais, oppo_static_pais, last_pai)
        elif is_chi(oppo_dynamic_pais, last_pai): # 吃
            chi(oppo_dynamic_pais, oppo_static_pais, last_pai)
        else: # 摸牌
            mopai(oppo_dynamic_pais)

        # 判断是否能胡牌
        if is_hu(oppo_dynamic_pais + oppo_static_pais):
            not_finished = False
            print("对手胡了: ", oppo_dynamic_pais + oppo_static_pais)
        else:# 出牌
            p = chu_pai(oppo_dynamic_pais)
            oppo_dynamic_pais.remove(p)
            oppo_output_pais.append(p)
        
        oppo_turn = False
    else:
        # 最后一次出牌
        last_pai = oppo_output_pais[-1]
        
        if is_gang(host_dynamic_pais, last_pai):# 杠
            gang(host_dynamic_pais, host_static_pais, last_pai)
        elif is_peng(host_dynamic_pais, last_pai): # 碰
            peng(host_dynamic_pais, host_static_pais, last_pai)
        elif is_chi(host_dynamic_pais, last_pai): # 吃
            chi(host_dynamic_pais, host_static_pais, last_pai)
        else: # 摸牌
            mopai(host_dynamic_pais)

        # 判断是否能胡牌
        if is_hu(host_dynamic_pais + host_static_pais):
            not_finished = False
            print("庄胡了: ", host_dynamic_pais + host_static_pais)
        else:# 出牌
            p = chu_pai(host_dynamic_pais)
            host_dynamic_pais.remove(p)
            host_output_pais.append(p)
        
        oppo_turn = True 

对手胡了:  [9, 7, 9, 12, 7, 9, 12, 12, 1, 2, 3, 2, 3, 4]


In [24]:
import collections

def mopai(my_pais):
    """
    摸牌，判断牌是否已经摸完
    """
    if len(pais) > 0:
        my_pais.append(pais.pop()) 
    else:
        print("流局")

def is_peng(my_dynamic_pais, pai):
    """
    判断是否可以碰对家出的牌
    """
    return my_dynamic_pais.count(pai) == 2

def peng(my_dynamic_pais, my_static_pais, pai):
    """
    碰牌且把相关牌从动态牌面中移除，添加到静态牌面
    """
   
    removeEle(my_dynamic_pais, pai)
    my_static_pais.extend([pai, pai, pai])

def is_chi(my_pais, pai):
    """
    判断是否可以吃对家出的牌
    """
    if pai <= 9 and pai - 2 in my_pais and pai - 1 in my_pais:
        return True
    elif pai <= 8 and pai - 1 in my_pais and pai + 1 in my_pais:
        return True
    elif pai <= 7 and pai + 1 in my_pais and pai + 2 in my_pais:
        return True
    return False

def chi(my_dynamic_pais, my_static_pais, pai):
    """
    吃牌且把相关牌从动态牌面中移除，添加到静态牌面
    """
    
    if pai <= 9 and  pai - 2 in my_dynamic_pais and pai - 1 in my_dynamic_pais:
        my_static_pais.extend([pai - 2, pai - 1, pai])
        my_dynamic_pais.remove(pai - 1)
        my_dynamic_pais.remove(pai - 2)
    elif pai<= 8 and pai - 1 in my_dynamic_pais and pai + 1 in my_dynamic_pais:
        my_static_pais.extend([pai - 1, pai, pai + 1])
        my_dynamic_pais.remove(pai - 1)
        my_dynamic_pais.remove(pai + 1)
    elif pai <=7 and pai + 1 in my_dynamic_pais and pai + 2 in my_dynamic_pais: 
        my_static_pais.extend([pai, pai + 1, pai + 2])
        my_dynamic_pais.remove(pai + 1)
        my_dynamic_pais.remove(pai + 2)  

def removeEle(pais, pai):
    """
    删除指定元素
    """
    for index, p in enumerate(pais):
        if p == pai:
            del pais[index]

def is_hu(my_pais):
    """
    判断是否可以胡牌，牌可以是自己摸的牌或胡对家的牌
    """
    
    # 每个牌出现的次数
    pai_count = {pai:0 for pai in pai_types.values()}
    for pai in my_pais:
        pai_count[pai] += 1
    
    # 所有可能的对子,计算每种胡的可能
    for i in pai_count.keys():
        if pai_count[i] >= 2:
            pai_count[i] -= 2;
            # 这里可以加上番数
            if check_sequence_triplet(pai_count):
                return True 
            pai_count[i] += 2;
    return False

def check_sequence_triplet(pai):
    # 判断是否所有元素已经使用，是：表明已经可以胡了
    success = True
    for i in pai.values():
        if i != 0:
            success = False
            break
    if success:
        return True
    
    # i 表示牌的编号
    for i in pai.keys():
        # 判断有没有顺子
        sunzi = i + 2 <= 9 and pai[i] >= 1 and pai[i + 1] >= 1 and pai[i + 2] >= 1
        if (sunzi):
            pai[i] -= 1
            pai[i + 1] -= 1
            pai[i + 2] -=1
            if check_sequence_triplet(pai):
                return True
            pai[i] += 1
            pai[i + 1] += 1
            pai[i + 2] += 1
        
        # 判断有没有刻子
        if pai[i] >= 3:
            pai[i] -= 3
            if check_sequence_triplet(pai):
                return True
            pai[i] += 3
    return False

def is_gang(my_pais, pai):
    """
    判断是否可以杠
    """
    return my_pais.count(pai) == 3

def gang(my_dynamic_pais, my_static_pais, pai):
    """
    杠
    """
    removeEle(my_dynamic_pais, pai)
    my_static_pais.extend([pai, pai, pai, pai])
    mopai(my_dynamic_pais)

def chu_pai(my_pais):
    """
    出牌逻辑，这部分待优化，出单牌
    """
    pai_count = collections.Counter(my_pais)
    for (k,v) in pai_count.items():
        if v == 1:
            return k
    return my_pais[0]

def cal_score():
    """
    计算牌的得分
    """
    pass